In [24]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
WIN = 'D:/'
MAC = '/Volumes/Transcend2/'

osp = MAC

## Load in limma results

In [8]:
toptable_ind = pd.read_csv(osp+'pymz/mzmine/221119/limma_toptable_independent.csv', sep=',')
feature_description = pd.read_csv(osp+'pymz/mzmine/221119/feature_description.csv', sep=',', index_col = 0)

In [11]:
#using the non-interaction model
data = feature_description.loc[toptable_ind.index]
#data['p-val'] = toptable_ind['P.Value']
data['adj p-val'] = toptable_ind['adj.P.Val']
data['t-score'] = toptable_ind['t']
data['custom_id'] = list(data.index)

In [ ]:
#save to csv for mummichog analysis
data.to_csv(osp+'pymz/mzmine/221119/data_for_mummichog.txt', header=True, index=None, sep='\t', mode='w')

## Run mummichog

In [14]:
os.system('mummichog -f'+ osp + 'pymz/mzmine/221119/data_for_mummichog.txt'+' -m positive -u 3 -k MummichogResults -o mm -c 0.05')

NameError: name 'os' is not defined

## Annotate data

In [15]:
compounds = pd.read_csv('MummichogResults/1582286605.18.mm/tables/ListOfEmpiricalCompounds.tsv', sep='\t')
featureoutput = pd.read_csv('MummichogResults/1582286605.18.mm/tables/userInputData.txt', sep='\t')
pathway = pd.read_csv('MummichogResults/1582286605.18.mm/tables/mcg_pathwayanalysis_mm.tsv', sep='\t')

In [16]:
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    matching_ids[key] = value

In [17]:
compounds_dict = {}
for _,row in compounds.iterrows():
    ids = row['massfeature_rows']
    ions = row['str_row_ion']
    compoundlist = row['compound_names']
    
    ids_list = ids.split(';')
    ions_list = ions.split(';')
    for i in range(len(ids_list)):
        compounds_dict[matching_ids[ids_list[i]]] = []
        compounds_dict[matching_ids[ids_list[i]]].append(ions_list[i].split('_')[1])
        compounds_dict[matching_ids[ids_list[i]]].append(compoundlist)
        

In [18]:
for row_id,row in data.iterrows():
    if row_id in list(compounds_dict.keys()):
        
        data.at[row_id,'compound_names'] = compounds_dict[row_id][-1]
    else:
         data.at[row_id,'compound_names'] = 'NaN'

In [21]:
ecomp_dict = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    massfeature_rows = row['massfeature_rows'].split(';')
    for mass in massfeature_rows:
        if mass in ecomp_dict:
            ecomp_dict[mass].append(eid)
        else:
            ecomp_dict[mass] = []
            ecomp_dict[mass].append(eid)
            
cmpd_names = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    name = row['compound_names']
    cmpd_names[eid] = name
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    
    matching_ids[value] = key
for n in matching_ids:
    if matching_ids[n] in ecomp_dict:
        matching_ids[n] = ecomp_dict[matching_ids[n]]

id_names = {}
for i in matching_ids:
    if type(matching_ids[i]) is list:
        for f in matching_ids[i]:
            if f in cmpd_names:
                if i in id_names:
                    id_names[i].append(cmpd_names[f])
                else:
                    id_names[i] = []
                    id_names[i].append(cmpd_names[f])
matching_ids2 = {}
for n in matching_ids:
    
    if type(matching_ids[n]) is list:
        lista = matching_ids[n]
        for p in lista:
            if p in matching_ids2:
                matching_ids2[p].append(n)
            else:
                matching_ids2[p] = []
                matching_ids2[p].append(n)
pathway_dict = {}
for _,row in pathway.iterrows():
    
    pname = row['pathway']
    idlist =row['overlap_EmpiricalCompounds (id)']
    
    if type(idlist) != float:
        idlist = idlist.split(',')
        for p in idlist:
            
            if pname in pathway_dict:
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
            else:
                pathway_dict[pname] = []
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
pathway_dict2 = {}
for p in pathway_dict:
    
    for n in pathway_dict[p]:
        for i in n:
            if i in pathway_dict2:
                if p not in  pathway_dict2[i]:
                    pathway_dict2[i].append(p)
            else:
                pathway_dict2[i] = []
                pathway_dict2[i].append(p)
                
data['compound_names'] = str(np.zeros(len(data)))
data['compound_id'] = str(np.zeros(len(data)))
data['pathway'] = str(np.zeros(len(data)))
for row_id,row in data.iterrows():
    if row_id in id_names:
        data.at[row_id,'compound_names'] = id_names[row_id]    
    else:
        data.at[row_id,'compound_names'] = 'NaN'
    if row_id in matching_ids:
        data.at[row_id,'compound_id'] = matching_ids[row_id]
    else:
        data.at[row_id,'compound_id'] = 'NaN'
    if row_id in pathway_dict2:
        data.at[row_id,'pathway'] = pathway_dict2[row_id]
    else:
        data.at[row_id,'pathway'] = 'NaN'
    

## Matching with standards ToxID annotation

In [25]:
zik_standards_file_list = glob.glob(os.path.join(osp,'Metabolomics_datasets/Zika/', "*Std*.csv"))
zik_standards_file_list.sort()

zik_std1_csv_file = pd.read_csv(zik_standards_file_list[0], skiprows=9, index_col=False)
zik_std2_csv_file = pd.read_csv(zik_standards_file_list[2], skiprows=15, index_col=False)
zik_std3_csv_file = pd.read_csv(zik_standards_file_list[4], skiprows=15, index_col=False)


In [26]:
from metab_utils import create_std_dict

zik_standards_1 = create_std_dict(zik_std1_csv_file)
zik_standards_2 = create_std_dict(zik_std2_csv_file)
zik_standards_3 = create_std_dict(zik_std3_csv_file)

from metab_utils import mergeDict
zik_standards = mergeDict(mergeDict(zik_standards_1,zik_standards_2), zik_standards_3)


In [27]:
#filtered_peaks['standards_match'] = ['']*len(filtered)
for i,row in data.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for metabolite in zik_standards:
        mz_std = zik_standards[metabolite][0]
        rt_std = zik_standards[metabolite][2]
        
        if mz >= mz_std - 0.001 and mz <= mz_std + 0.001:
            if rt >= rt_std - 0.2 and rt <= rt_std + 0.2:
                data.at[i,'standards_match'] = metabolite
            else:
                data.at[i,'standards_match'] = 'NaN'
    

In [29]:
data.to_csv(osp+'pymz/mzmine/221119/annotated_data.csv', sep = ',')

## Check accuracy of mummichog annotation

In [ ]:
notannotated = 0
annotated = 0
notannotateds = 0
annotateds = 0
match = 0
matchs =0
for i,row in filtered_peaks.iterrows():
    cmp = str(row['compound_names']).lower()
    std = str(row['standards_match']).lower()
    
    if cmp in ['nan']:
        notannotated +=1
        if not std in ['nan']:
            matchs +=1
           
    else:
        annotated +=1
        if not std in ['nan']:
            if std in cmp:
                match +=1
            else:
                print(row)
    
    if std in ['nan']:
        notannotateds +=1
    else:
        annotateds +=1
        
percentage_exact_matches = match*100/annotateds
percentage_not_annotated = matchs*100/annotateds
percentage_not_matched = (annotateds-match-matchs)*100/annotateds

print("Total number of annotated compounds using mummichog: {}".format(annotated))
print("Total number of annotated compounds using ToxID standards: {}".format(annotateds))
print("Total number of annotation matches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(match, percentage_exact_matches))
print("Total number of annotation mismatches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(annotateds-match-matchs, percentage_not_matched))
print("Total number of metabolites annotated with ToxID, but not with mummichog: {0}, i.e. {1:.2f}%".format(matchs, percentage_not_annotated))